In [3]:
import os
import random
import shutil
from typing import List, Dict, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F  # Добавлен импорт
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Конфигурация
class Config:
    MODEL_NAME = 'resnet18'
    PRETRAINED = True
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001
    BATCH_SIZE = 32
    IMG_SIZE = (224, 224)

class SimpleCNN(nn.Module):
    """Простая CNN модель для классификации масок"""
    
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        
        # Сверточные слои
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Max pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Полносвязные слои
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # 224/2/2/2 = 28
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)
        
        # Dropout для регуляризации
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # Сверточные слои с активацией и пуллингом
        x = self.pool(F.relu(self.conv1(x)))  # 224x224 -> 112x112
        x = self.pool(F.relu(self.conv2(x)))  # 112x112 -> 56x56
        x = self.pool(F.relu(self.conv3(x)))  # 56x56 -> 28x28
        
        # Выравнивание для полносвязных слоев
        x = x.view(-1, 128 * 28 * 28)
        
        # Полносвязные слои
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

class SimpleMaskClassifier:
    """Классификатор на основе простой CNN"""
    
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SimpleCNN().to(self.device)
        self.transform = self._get_transforms()
        
    def _get_transforms(self):
        """Возвращает трансформации для изображений"""
        return {
            'train': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        }
    
    def train(self, train_loader: DataLoader, num_epochs: int = Config.NUM_EPOCHS):
        """Обучение модели"""
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=Config.LEARNING_RATE)
        
        print(f"Начало обучения простой CNN модели...")
        print(f"Количество эпох: {num_epochs}")
        print(f"Размер батча: {Config.BATCH_SIZE}")
        print(f"Устройство: {self.device}")
        
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            
            for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            
            train_acc = 100 * correct / total
            train_loss = running_loss / len(train_loader)
            
            print(f"Epoch {epoch+1}/{num_epochs}, "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        
        print(f"Обучение простой CNN завершено! Финальная точность: {train_acc:.2f}%")
    
    def classify_and_organize(self, test_data: List[Dict], output_dir: str = "mask_classification_results"):
        """
        Классифицирует изображения и распределяет по папкам
        WithMask - человек в маске
        WithoutMask - человек без маски
        """
        self.model.eval()
        
        # Создаем папки для результатов
        with_mask_dir = os.path.join(output_dir, "WithMask")
        without_mask_dir = os.path.join(output_dir, "WithoutMask")
        os.makedirs(with_mask_dir, exist_ok=True)
        os.makedirs(without_mask_dir, exist_ok=True)
        
        with_mask_count = 0
        without_mask_count = 0
        
        with torch.no_grad():
            for item in tqdm(test_data, desc="Классификация изображений (SimpleCNN)"):
                try:
                    # Преобразуем изображение
                    image = Image.fromarray(item['image'])
                    tensor = self.transform['test'](image).unsqueeze(0).to(self.device)
                    
                    # Предсказание
                    outputs = self.model(tensor)
                    _, predicted = torch.max(outputs, 1)
                    
                    # Определяем класс
                    if predicted.item() == 0:  # WithMask
                        dest_dir = with_mask_dir
                        with_mask_count += 1
                    else:  # WithoutMask
                        dest_dir = without_mask_dir
                        without_mask_count += 1
                    
                    # Копируем файл в соответствующую папку
                    src_path = item['filepath']
                    dst_path = os.path.join(dest_dir, item['filename'])
                    shutil.copy2(src_path, dst_path)
                    
                except Exception as e:
                    print(f"Ошибка обработки {item['filename']}: {str(e)}")
        
        print(f"\nКлассификация завершена!")
        print(f"Результаты сохранены в папке: {output_dir}")
        print(f"WithMask (в маске): {with_mask_count} изображений")
        print(f"WithoutMask (без маски): {without_mask_count} изображений")
    
    def classify_single_image(self, image_path: str):
        """
        Классифицирует одно изображение и выводит результат
        """
        self.model.eval()
        
        if not os.path.exists(image_path):
            print(f"Ошибка: файл {image_path} не найден")
            return
        
        try:
            # Загружаем и обрабатываем изображение
            with Image.open(image_path) as img:
                img = img.convert('RGB')
                img = img.resize(Config.IMG_SIZE)
                tensor = self.transform['test'](img).unsqueeze(0).to(self.device)
            
            # Предсказание
            with torch.no_grad():
                outputs = self.model(tensor)
                probabilities = torch.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)
                
                class_name = "WithMask" if predicted.item() == 0 else "WithoutMask"
                confidence = probabilities[0][predicted.item()].item() * 100
                
                print(f"\nРезультат классификации (SimpleCNN):")
                print(f"Изображение: {os.path.basename(image_path)}")
                print(f"Класс: {class_name}")
                print(f"Уверенность: {confidence:.2f}%")
                
                # Показываем изображение
                plt.figure(figsize=(8, 6))
                plt.imshow(np.array(img))
                plt.title(f"Результат: {class_name} ({confidence:.2f}%) - SimpleCNN")
                plt.axis('off')
                plt.show()
                
        except Exception as e:
            print(f"Ошибка обработки изображения: {str(e)}")
    
    def save_model(self, filepath: str):
        """Сохраняет модель"""
        torch.save({
            'model_state_dict': self.model.state_dict(),
        }, filepath)
        print(f"Простая CNN модель сохранена в {filepath}")
    
    def load_model(self, filepath: str):
        """Загружает модель"""
        checkpoint = torch.load(filepath, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Простая CNN модель загружена из {filepath}")

class FaceMaskDataset:
    """
    Загрузка изображений для классификации масок
    """
    def __init__(self, root_dir: str, img_size: Tuple[int, int] = (224, 224), mode: str = 'train'):
        self.root_dir = root_dir
        self.img_size = img_size
        self.mode = mode
        self.dataset = []
        self.class_to_idx = {'WithMask': 0, 'WithoutMask': 1}
        self.idx_to_class = {0: 'WithMask', 1: 'WithoutMask'}
        
    def load(self) -> List[Dict]:
        """Основной метод загрузки данных"""
        if self.mode == 'train':
            return self._load_train_data()
        else:
            return self._load_test_data()
    
    def _load_train_data(self) -> List[Dict]:
        """Загрузка тренировочных данных"""
        for class_name in ['WithMask', 'WithoutMask']:
            class_dir = os.path.join(self.root_dir, class_name)
            if not os.path.isdir(class_dir):
                print(f"Предупреждение: папка {class_dir} не найдена")
                continue
                
            for filename in os.listdir(class_dir):
                if self._is_image_file(filename):
                    filepath = os.path.join(class_dir, filename)
                    try:
                        img = self._process_image(filepath)
                        if img is not None:
                            self.dataset.append({
                                'image': img,
                                'class_idx': self.class_to_idx[class_name],
                                'class_name': class_name,
                                'filename': filename,
                                'filepath': filepath
                            })
                    except Exception as e:
                        print(f"Ошибка загрузки {filepath}: {str(e)}")
        
        print(f"Загружено {len(self.dataset)} изображений")
        return self.dataset
    
    def _load_test_data(self) -> List[Dict]:
        """Загрузка тестовых данных"""
        for filename in os.listdir(self.root_dir):
            filepath = os.path.join(self.root_dir, filename)
            if not os.path.isfile(filepath):
                continue
                
            if self._is_image_file(filename):
                try:
                    img = self._process_image(filepath)
                    self.dataset.append({
                        'image': img,
                        'class_idx': -1,
                        'class_name': 'unknown',
                        'filename': filename,
                        'filepath': filepath
                    })
                except Exception as e:
                    print(f"Ошибка загрузки {filepath}: {str(e)}")
        
        print(f"Загружено {len(self.dataset)} тестовых изображений")
        return self.dataset
    
    def _process_image(self, path: str) -> np.ndarray:
        """Загружает и обрабатывает изображение"""
        try:
            with Image.open(path) as img:
                img = img.convert('RGB')
                img = img.resize(self.img_size)
                return np.array(img)
        except Exception as e:
            print(f"Ошибка обработки {path}: {str(e)}")
            return None
    
    def _is_image_file(self, filename: str) -> bool:
        """Проверяет, является ли файл изображением"""
        return filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))

class CustomMaskDataset(Dataset):
    """Датасет для изображений с масками"""
    
    def __init__(self, data, transform=None, augment: bool = False):
        self.data = data
        self.transform = transform
        self.augment = augment
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_data = self.data[idx]  # Исправлена опечатка: было self.data[idx)
        img_array = img_data['image']
        label = img_data['class_idx']
        
        img = Image.fromarray(img_array)
        
        if self.augment:
            # Простые аугментации
            if random.random() < 0.5:
                img = transforms.functional.hflip(img)
            if random.random() < 0.3:
                img = transforms.functional.rotate(img, angle=random.uniform(-15, 15))
        
        if self.transform:
            img = self.transform(img)
            
        return img, label

class MaskClassifier:
    """Классификатор для определения масок на лицах"""
    
    def __init__(self, model_name: str = Config.MODEL_NAME, pretrained: bool = Config.PRETRAINED):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._initialize_model(model_name, pretrained)
        self.transform = self._get_transforms()
        
    def _initialize_model(self, model_name: str, pretrained: bool) -> nn.Module:
        """Инициализирует модель"""
        model_func = getattr(models, model_name)
        model = model_func(pretrained=pretrained)
        
        # Заменяем последний слой для бинарной классификации
        if hasattr(model, 'fc'):
            model.fc = nn.Linear(model.fc.in_features, 2)
        elif hasattr(model, 'classifier'):
            if isinstance(model.classifier, nn.Linear):
                model.classifier = nn.Linear(model.classifier.in_features, 2)
        
        return model.to(self.device)
    
    def _get_transforms(self) -> Dict[str, transforms.Compose]:
        """Возвращает трансформации для изображений"""
        return {
            'train': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        }
    
    def train(self, train_loader: DataLoader, num_epochs: int = Config.NUM_EPOCHS):
        """Обучение модели"""
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=Config.LEARNING_RATE)
        
        print(f"Начало обучения ResNet модели...")
        print(f"Тип модели: {Config.MODEL_NAME}")
        print(f"Количество эпох: {num_epochs}")
        print(f"Размер батча: {Config.BATCH_SIZE}")
        print(f"Устройство: {self.device}")
        
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            
            for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            
            train_acc = 100 * correct / total
            train_loss = running_loss / len(train_loader)
            
            print(f"Epoch {epoch+1}/{num_epochs}, "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        
        print(f"Обучение ResNet завершено! Финальная точность: {train_acc:.2f}%")
    
    def classify_and_organize(self, test_data: List[Dict], output_dir: str = "mask_classification_results"):
        """
        Классифицирует изображения и распределяет по папкам
        WithMask - человек в маске
        WithoutMask - человек без маски
        """
        self.model.eval()
        
        # Создаем папки для результатов
        with_mask_dir = os.path.join(output_dir, "WithMask")
        without_mask_dir = os.path.join(output_dir, "WithoutMask")
        os.makedirs(with_mask_dir, exist_ok=True)
        os.makedirs(without_mask_dir, exist_ok=True)
        
        with_mask_count = 0
        without_mask_count = 0
        
        with torch.no_grad():
            for item in tqdm(test_data, desc="Классификация изображений"):
                try:
                    # Преобразуем изображение
                    image = Image.fromarray(item['image'])
                    tensor = self.transform['test'](image).unsqueeze(0).to(self.device)
                    
                    # Предсказание
                    outputs = self.model(tensor)
                    _, predicted = torch.max(outputs, 1)
                    
                    # Определяем класс
                    if predicted.item() == 0:  # WithMask
                        dest_dir = with_mask_dir
                        with_mask_count += 1
                    else:  # WithoutMask
                        dest_dir = without_mask_dir
                        without_mask_count += 1
                    
                    # Копируем файл в соответствующую папку
                    src_path = item['filepath']
                    dst_path = os.path.join(dest_dir, item['filename'])
                    shutil.copy2(src_path, dst_path)
                    
                except Exception as e:
                    print(f"Ошибка обработки {item['filename']}: {str(e)}")
        
        print(f"\nКлассификация завершена!")
        print(f"Результаты сохранены в папке: {output_dir}")
        print(f"WithMask (в маске): {with_mask_count} изображений")
        print(f"WithoutMask (без маски): {without_mask_count} изображений")
    
    def classify_single_image(self, image_path: str):
        """
        Классифицирует одно изображение и выводит результат
        """
        self.model.eval()
        
        if not os.path.exists(image_path):
            print(f"Ошибка: файл {image_path} не найден")
            return
        
        try:
            # Загружаем и обрабатываем изображение
            with Image.open(image_path) as img:
                img = img.convert('RGB')
                img = img.resize(Config.IMG_SIZE)
                tensor = self.transform['test'](img).unsqueeze(0).to(self.device)
            
            # Предсказание
            with torch.no_grad():
                outputs = self.model(tensor)
                probabilities = torch.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)
                
                class_name = "WithMask" if predicted.item() == 0 else "WithoutMask"
                confidence = probabilities[0][predicted.item()].item() * 100
                
                print(f"\nРезультат классификации (ResNet):")
                print(f"Изображение: {os.path.basename(image_path)}")
                print(f"Класс: {class_name}")
                print(f"Уверенность: {confidence:.2f}%")
                
                # Показываем изображение
                plt.figure(figsize=(8, 6))
                plt.imshow(np.array(img))
                plt.title(f"Результат: {class_name} ({confidence:.2f}%) - ResNet")
                plt.axis('off')
                plt.show()
                
        except Exception as e:
            print(f"Ошибка обработки изображения: {str(e)}")
    
    def save_model(self, filepath: str):
        """Сохраняет модель"""
        torch.save({
            'model_state_dict': self.model.state_dict(),
        }, filepath)
        print(f"ResNet модель сохранена в {filepath}")
    
    def load_model(self, filepath: str):
        """Загружает модель"""
        checkpoint = torch.load(filepath, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        print(f"ResNet модель загружена из {filepath}")

def train_resnet_model():
    """Обучение ResNet модели с нуля"""
    print("=== ОБУЧЕНИЕ RESNET МОДЕЛИ ===")
    
    # Загрузка тренировочных данных
    train_dataset = FaceMaskDataset(
        root_dir=r"C:\Users\USER\Desktop\Face Mask Dataset\Train",
        img_size=Config.IMG_SIZE,
        mode='train'
    )
    train_data = train_dataset.load()
    
    # Подготовка DataLoader
    train_custom_dataset = CustomMaskDataset(
        train_data, 
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]), 
        augment=True
    )
    
    train_loader = DataLoader(train_custom_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    
    # Обучение модели
    classifier = MaskClassifier()
    classifier.train(train_loader)
    
    # Сохранение модели
    classifier.save_model("resnet_face_mask_model.pth")
    
    return classifier

def train_simple_cnn_model():
    """Обучение простой CNN модели с нуля"""
    print("=== ОБУЧЕНИЕ ПРОСТОЙ CNN МОДЕЛИ ===")
    
    # Загрузка тренировочных данных
    train_dataset = FaceMaskDataset(
        root_dir=r"C:\Users\USER\Desktop\Face Mask Dataset\Train",
        img_size=Config.IMG_SIZE,
        mode='train'
    )
    train_data = train_dataset.load()
    
    # Подготовка DataLoader
    train_custom_dataset = CustomMaskDataset(
        train_data, 
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]), 
        augment=True
    )
    
    train_loader = DataLoader(train_custom_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    
    # Обучение модели
    classifier = SimpleMaskClassifier()
    classifier.train(train_loader)
    
    # Сохранение модели
    classifier.save_model("simple_cnn_face_mask_model.pth")
    
    return classifier

def classify_images():
    """Классификация изображений и распределение по папкам"""
    print("=== КЛАССИФИКАЦИЯ ИЗОБРАЖЕНИЙ ===")
    
    # Выбор модели
    print("Выберите модель для классификации:")
    print("1. ResNet модель")
    print("2. Простая CNN модель")
    choice = input("Введите 1 или 2: ").strip()
    
    if choice == "1":
        model_path = "resnet_face_mask_model.pth"
        classifier = MaskClassifier()
    elif choice == "2":
        model_path = "simple_cnn_face_mask_model.pth"
        classifier = SimpleMaskClassifier()
    else:
        print("Неверный выбор, используется ResNet модель по умолчанию")
        model_path = "resnet_face_mask_model.pth"
        classifier = MaskClassifier()
    
    # Проверка существования модели
    if not os.path.exists(model_path):
        print(f"Ошибка: файл модели {model_path} не найден!")
        print("Сначала обучите модель.")
        return
    
    # Загрузка тестовых данных
    test_dataset = FaceMaskDataset(
        root_dir=r"C:\Users\USER\Desktop\Face Mask Dataset\Test",
        img_size=Config.IMG_SIZE,
        mode='test'
    )
    test_data = test_dataset.load()
    
    # Загрузка модели
    classifier.load_model(model_path)
    
    # Классификация и распределение по папкам
    output_dir = f"classification_results_{'resnet' if choice == '1' else 'simple_cnn'}"
    classifier.classify_and_organize(test_data, output_dir)

def classify_single_image():
    """Классификация одного изображения"""
    print("=== КЛАССИФИКАЦИЯ ОДНОГО ИЗОБРАЖЕНИЯ ===")
    
    image_path = input("Введите путь к изображению: ").strip()
    
    if not os.path.exists(image_path):
        print("Ошибка: файл не существует")
        return
    
    # Выбор модели
    print("Выберите модель для классификации:")
    print("1. ResNet модель")
    print("2. Простая CNN модель")
    choice = input("Введите 1 или 2: ").strip()
    
    if choice == "1":
        model_path = "resnet_face_mask_model.pth"
        classifier = MaskClassifier()
    elif choice == "2":
        model_path = "simple_cnn_face_mask_model.pth"
        classifier = SimpleMaskClassifier()
    else:
        print("Неверный выбор, используется ResNet модель по умолчанию")
        model_path = "resnet_face_mask_model.pth"
        classifier = MaskClassifier()
    
    if not os.path.exists(model_path):
        print(f"Ошибка: файл модели {model_path} не найден")
        print("Сначала обучите модель.")
        return
    
    # Загрузка модели и классификация
    classifier.load_model(model_path)
    classifier.classify_single_image(image_path)

def evaluate_accuracy():
    """
    Сравнивает результаты классификации с правильным распределением
    и вычисляет точность в процентах
    """
    print("=== ОЦЕНКА ТОЧНОСТИ КЛАССИФИКАЦИИ ===")
    
    # Выбор папки с результатами
    print("Выберите папку с результатами для оценки:")
    print("1. classification_results_resnet (ResNet)")
    print("2. classification_results_simple_cnn (Simple CNN)")
    choice = input("Введите 1 или 2: ").strip()
    
    if choice == "1":
        results_dir = "classification_results_resnet"
    elif choice == "2":
        results_dir = "classification_results_simple_cnn"
    else:
        print("Неверный выбор, используется ResNet результаты по умолчанию")
        results_dir = "classification_results_resnet"
    
    ground_truth_dir = r"C:\Users\USER\Desktop\Face Mask Dataset\Test_r"
    
    if not os.path.exists(results_dir):
        print(f"Ошибка: папка с результатами {results_dir} не найдена")
        return
    
    if not os.path.exists(ground_truth_dir):
        print(f"Ошибка: папка с правильными ответами {ground_truth_dir} не найдена")
        return
    
    # Собираем информацию о правильной классификации
    ground_truth = {}
    for class_name in ['WithMask', 'WithoutMask']:
        class_dir = os.path.join(ground_truth_dir, class_name)
        if os.path.exists(class_dir):
            for filename in os.listdir(class_dir):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    ground_truth[filename] = class_name
    
    # Собираем информацию о нашей классификации
    our_results = {}
    for class_name in ['WithMask', 'WithoutMask']:
        class_dir = os.path.join(results_dir, class_name)
        if os.path.exists(class_dir):
            for filename in os.listdir(class_dir):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    our_results[filename] = class_name
    
    # Сравниваем результаты
    correct = 0
    total = 0
    mismatched_files = []
    
    for filename, true_class in ground_truth.items():
        if filename in our_results:
            total += 1
            if our_results[filename] == true_class:
                correct += 1
            else:
                mismatched_files.append((filename, true_class, our_results[filename]))
    
    # Вычисляем точность
    if total > 0:
        accuracy = (correct / total) * 100
        print(f"\nРезультаты оценки точности:")
        print(f"Модель: {'ResNet' if choice == '1' else 'Simple CNN'}")
        print(f"Всего изображений: {total}")
        print(f"Правильно классифицировано: {correct}")
        print(f"Неправильно классифицировано: {len(mismatched_files)}")
        print(f"Точность: {accuracy:.2f}%")
        
        if mismatched_files:
            print(f"\nНеправильно классифицированные файлы:")
            for filename, true_class, our_class in mismatched_files[:10]:  # Показываем первые 10
                print(f"  {filename}: должно быть {true_class}, наш результат {our_class}")
            if len(mismatched_files) > 10:
                print(f"  ... и еще {len(mismatched_files) - 10} файлов")
    else:
        print("Нет файлов для сравнения")

def main():
    """Основное меню программы"""
    while True:
        print("\n" + "="*50)
        print("Классификатор масок на лицах")
        print("="*50)
        print("1. Обучить ResNet модель")
        print("2. Обучить простую CNN модель")
        print("3. Классифицировать все тестовые изображения")
        print("4. Классифицировать одно изображение")
        print("5. Оценить точность классификации")
        print("6. Выход")
        
        choice = input("\nВыберите действие (1-6): ").strip()
        
        if choice == "1":
            train_resnet_model()
            print("\nХотите сразу классифицировать тестовые изображения? (y/n)")
            if input().lower() == 'y':
                classify_images()
                
        elif choice == "2":
            train_simple_cnn_model()
            print("\nХотите сразу классифицировать тестовые изображения? (y/n)")
            if input().lower() == 'y':
                classify_images()
                
        elif choice == "3":
            classify_images()
            
        elif choice == "4":
            classify_single_image()
            
        elif choice == "5":
            evaluate_accuracy()
            
        elif choice == "6":
            print("Выход из программы")
            break
            
        else:
            print("Неверный выбор. Пожалуйста, выберите от 1 до 6.")

if __name__ == "__main__":
    main()


Классификатор масок на лицах
1. Обучить ResNet модель
2. Обучить простую CNN модель
3. Классифицировать все тестовые изображения
4. Классифицировать одно изображение
5. Оценить точность классификации
6. Выход



Выберите действие (1-6):  2


=== ОБУЧЕНИЕ ПРОСТОЙ CNN МОДЕЛИ ===
Загружено 10000 изображений
Начало обучения простой CNN модели...
Количество эпох: 10
Размер батча: 32
Устройство: cpu


Epoch 1/10: 100%|██████████| 313/313 [06:30<00:00,  1.25s/it]


Epoch 1/10, Train Loss: 0.1669, Train Acc: 93.99%


...

=== ОЦЕНКА ТОЧНОСТИ КЛАССИФИКАЦИИ ===
Выберите папку с результатами для оценки:
1. classification_results_resnet (ResNet)
2. classification_results_simple_cnn (Simple CNN)


Введите 1 или 2:  2



Результаты оценки точности:
Модель: Simple CNN
Всего изображений: 992
Правильно классифицировано: 986
Неправильно классифицировано: 6
Точность: 99.40%

Неправильно классифицированные файлы:
  Augmented_187_5733934_y.png: должно быть WithMask, наш результат WithoutMask
  Augmented_353_5799649_y.png: должно быть WithMask, наш результат WithoutMask
  Augmented_689_9677847_y.png: должно быть WithMask, наш результат WithoutMask
  Augmented_814_3182702_y.png: должно быть WithMask, наш результат WithoutMask
  Augmented_890_8528242_y.png: должно быть WithMask, наш результат WithoutMask
  3372_n.png: должно быть WithoutMask, наш результат WithMask

Классификатор масок на лицах
1. Обучить ResNet модель
2. Обучить простую CNN модель
3. Классифицировать все тестовые изображения
4. Классифицировать одно изображение
5. Оценить точность классификации
6. Выход



Выберите действие (1-6):  6


Выход из программы
